# Kurszuteilung

## Implementierung

In [29]:
import random
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
# Python program to find maximal Bipartite matching.

class GFG:
	def __init__(self, graph: list[list[int]]):
		
		# residual graph
		self.graph = graph
		self.ppl = len(graph)
		self.jobs = len(graph[0])


	# A DFS based recursive function
	# that returns true if a matching
	# for vertex u is possible
	def bpm(self, u, matchR, seen):

		# Try every job one by one
		for v in range(self.jobs):

			# If applicant u is interested
			# in job v and v is not seen
			if self.graph[u][v] and seen[v] == False:
				
				# Mark v as visited
				seen[v] = True

				'''If job 'v' is not assigned to
				an applicant OR previously assigned
				applicant for job v (which is matchR[v])
				has an alternate job available.
				Since v is marked as visited in the
				above line, matchR[v] in the following
				recursive call will not get job 'v' again'''
				if matchR[v] == -1 or self.bpm(matchR[v],
											matchR, seen):
					matchR[v] = u
					return True
		return False

	# Returns maximum number of matching
	def maxBPM(self):
		'''
		Returns List where each value at index i
		is the applicant index number that is
		assigned to job i.
		'''

		'''An array to keep track of the
		applicants assigned to jobs.
		The value of matchR[i] is the
		applicant number assigned to job i,
		the value -1 indicates nobody is assigned.'''
		matchR = [-1] * self.jobs
		
		for i in range(self.ppl):
			
			# Mark all jobs as not seen for next applicant.
			seen = [False] * self.jobs
			
			# Find if the applicant 'u' can get a job
			self.bpm(i, matchR, seen)
		return matchR

	# Takes a matching and assigns unmatched people to random open jobs
	def distribute_rest_to_open_jobs(self, matching_vector):
		# Filter for unassigned people
		set_matching_vector = set(matching_vector)
		set_ppl = set(range(self.ppl))
		not_assigned_ppl = list(set_ppl - set_matching_vector)
		freie_plaetze = [index for index in range(len(matching_vector)) if matching_vector[index] == -1]

		# Assign the unassigned people to a random open Job in random order
		while len(not_assigned_ppl)>0:
			rand_person = not_assigned_ppl.pop(random.randint(0,len(not_assigned_ppl)-1))
			rand_platz = freie_plaetze.pop(random.randint(0,len(freie_plaetze)-1))
			matching_vector[rand_platz] = rand_person
		
		return matching_vector


# This code is contributed by Neelam Yadav
# https://www.geeksforgeeks.org/maximum-bipartite-matching/


In [31]:
class Verteilung:
    def __init__(self, wahlergebnisse: str, input_name_format: int, kurse_dict: dict, *vollst_sus_listen: str):
        '''
            Arguments:
                input_name_format: 1 Wenn Vor- und Nachname in einer Spalte stehen, 2, wenn Vor- und Nachname einzeln gegeben sind.
        '''
        if input_name_format not in [1,2]:
            raise Exception("input_name_format nicht 1 oder 2. Gib 1 an, wenn Vor- und Nachname in einer Spalte stehen oder 2, wenn Vor- und Nachname einzeln gegeben sind.")
        self.input_name_format = input_name_format

        # Lese Datei-Inhalte
        self.df_gesamt = pd.read_csv(wahlergebnisse, delimiter=';', encoding='utf-8')

        # Wenn eine vollständige Schüler:innenliste gegeben ist, füge fehlende Schüeler:innen hinzu
        self._add_missing_students(*vollst_sus_listen)
        
        
        if sum(kurse_dict.values()) < len(self.df_gesamt):
            raise Exception("Verteilung von {} Schüler:innen auf {} Plätze nicht möglich".format(len(self.df_gesamt), sum(kurse_dict.values())))

        # DataFrame ohne Name und Klasse, also nur mit Kursen.
        # Dieser DataFrame dient zur bestimmung der Verteilung.
        # Erst im output werden die Namen und Klassen wieder zugeordnet.
        if self.input_name_format == 1:
            self.df_kurswahlen = self.df_gesamt[self.df_gesamt.columns[2:]]
        else:
            self.df_kurswahlen = self.df_gesamt[self.df_gesamt.columns[3:]]

        self.kurse_mit_limits = kurse_dict
        self.graph = GFG(self.make_adj_matrix().values.tolist())

        self.matching_vector = None

    def _add_missing_students(self, *vollst_sus_listen: str):
        if self.input_name_format == 1:
            # Ersetze umlaute in jeder Liste für ein einheitliches Format, um Duplikate besser zu erkennen.
            self.df_gesamt["Name"] = self.df_gesamt["Name"].replace({'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}, regex=True)
            for file in vollst_sus_listen:
                vollst_sus_liste = pd.read_csv(file, encoding='utf-8')
                vollst_sus_liste["Name"] = vollst_sus_liste["Vorname"] + " " +  vollst_sus_liste["Nachname"]
                vollst_sus_liste["Name"] = vollst_sus_liste["Name"].replace({'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}, regex=True)

                self.df_gesamt = pd.concat([self.df_gesamt, vollst_sus_liste],
                                            join="outer",
                                            ignore_index=True)\
                                .drop_duplicates(subset=['Name'], keep='first')\
                                [self.df_gesamt.columns]
        else:
            # Ersetze umlaute in jeder Liste für ein einheitliches Format, um Duplikate besser zu erkennen.
            self.df_gesamt["Vorname"] = self.df_gesamt["Vorname"].replace({'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}, regex=True)
            self.df_gesamt["Nachname"] = self.df_gesamt["Nachname"].replace({'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}, regex=True)
            for file in vollst_sus_listen:
                vollst_sus_liste = pd.read_csv(file, encoding='utf-8')
                vollst_sus_liste["Name"] = vollst_sus_liste["Vorname"] + " " +  vollst_sus_liste["Nachname"]
                vollst_sus_liste["Name"] = vollst_sus_liste["Name"].replace({'ä': 'ae', 'ö': 'oe', 'ü': 'ue'}, regex=True)

                self.df_gesamt = pd.concat([self.df_gesamt, vollst_sus_liste],
                                            join="outer",
                                            ignore_index=True)\
                                .drop_duplicates(subset=['Vorname', 'Nachname'], keep='first')\
                                [self.df_gesamt.columns]


    def make_adj_matrix(self) -> pd.DataFrame:
        df_adj_matrix = self.df_kurswahlen.copy()

        # Füge für jeden Kurs die Anzahl an Plätzen hinzu
        for kurs, limit in self.kurse_mit_limits.items():
            df_kurs_gewählt = self.df_kurswahlen.eq(kurs).any(axis=1)
            for i in range(limit):
                df_adj_matrix[f"{kurs}_platz_{i+1}"] = df_kurs_gewählt.astype(int)

        # Lösche die ursprünglichen Kurse, da diese durch die Plätze repräsentiert werden
        df_adj_matrix.drop(self.df_kurswahlen.columns[:], axis=1, inplace = True)

        return df_adj_matrix


    def erstellen(self, rest_zufaellig_verteilen = True) -> list[int]:
        # Berechne das maximum bipartite matching und verteile ungematchte Schüler zufällig auf offene Plätze
        self.matching_vector =  self.graph.maxBPM()
        if rest_zufaellig_verteilen:
            self.matching_vector = self.graph.distribute_rest_to_open_jobs(self.matching_vector)

        return self.matching_vector

        
    def get_kurs(self, platz_idx: int) -> str:
        fortlaufende_plaetze_fuer_kurse = 0
        for kurs, limit in self.kurse_mit_limits.items():
            if platz_idx <= fortlaufende_plaetze_fuer_kurse + limit - 1:
                return kurs
            fortlaufende_plaetze_fuer_kurse += limit
        raise Exception('Kurs konnte nicht zugeteilt werden. Platz Index {} liegt außerhalb der summierten Platz-Limits aller Kurse: {}'.format(platz_idx, fortlaufende_plaetze_fuer_kurse))


    def maybe_get_kurs_by_schueler_idx(self, row) -> str:
        if row.name in self.matching_vector:
            platz_idx =  self.matching_vector.index(row.name)
            kurs = self.get_kurs(platz_idx)
            return kurs
        return None


    # Erstellt output DataFrame
    def to_dataframe(self) -> pd.DataFrame:
        # Dataframe Spalten Nachname, Vorname, Klasse und Kurs
        df_output = self.df_gesamt.loc[:, ["Klasse"]]

        # Entferne Punkte aus Klassenbezeichnung, damit Excel daraus kein Datum macht
        df_output["Klasse"] = df_output["Klasse"].str.replace(".","")

        if self.input_name_format == 1:
            # Aufteilen der Spalte Name in Vor- und Nachname.
            # Nutze `str.rsplit()` um den ersten Space von rechts zu suchen, da jeder nur einen Nachnamen, aber mehrere Vornamen haben kann.
            df_output[['Vorname', 'Nachname']] = self.df_gesamt['Name'].str.rsplit(' ', n=1, expand=True)
        else:
            df_output['Vorname'] = self.df_gesamt['Vorname']
            df_output['Nachname'] = self.df_gesamt['Nachname']

        # Finde zu den Schüler:innen nach index im gesamten Namens-Dataframe den im Matchingvektor stehenden
        # Kursplatz und übersetze diesen wiederum in einen Kursnamen, anhand des kurse_mit_limits dict
        # Gibt `None` für Schüler:innen zurück, die keinem Platz zugeteilt sind.
        df_output['Kurs'] = df_output.apply(lambda row: self.maybe_get_kurs_by_schueler_idx(row), axis=1)

        # Ordne die Spalten wie in Excel Tabelle zum Erstellen der Kurs- und Klassenlisten
        df_output = df_output[['Nachname', 'Vorname', 'Klasse', 'Kurs']]

        return df_output
    
    def to_csv(self, filename: str) -> None:
        return self.to_dataframe().to_csv(filename, index=False, encoding='utf-8')

    
    def info(self) -> None:
        # Anzahl verteilter Schüler und offener Plätze
        offene_plaetze = self.matching_vector.count(-1)
        print(f"{len(self.matching_vector)-offene_plaetze} Schüler:innen von {self.graph.ppl} Schüler:innen verteilt.")
        print("-" * 50)

        output = self.to_dataframe()
        print("Belegung:")
        print(output.groupby("Kurs").count()["Klasse"])

        print("-" * 50)
        print("Plätze übrig: ", offene_plaetze)

### TODO

In [32]:
# TODO: Zur Zeit ist die Verteilung auf Kurse stark von der Reihenfolge der Kurse im dictionary abhängig.
#       Hier automatische Durchführung von Durchgängen mit unterschiedlicher Reihenfolge und Ermitteln der Reihenfolge
#       mit der geringsten Varianz der vergebenen Kursplätze um das jeweilige Limit.
# TODO: Strip trailing whitespace from input data (auch Spaltennamen in Wahlergebnissen und Projektnamen im dict und in Wahl-Spalten)
# TODO: Checken, ob Name aus UCS liste substring von dem in der Wahl ist und insb. anders herum, um Duplikate zu vermeiden.

## Daten Eingabe

### Format der eingelesenen csv Datei:

|Name|Klasse|Wahl_1|Wahl_2|Wahl_3|
|:--:|:----:|:----:|:----:|:----:|

oder

|Vorname|Nachname|Klasse|Wahl_1|Wahl_2|Wahl_3|
|:-----:|:------:|:----:|:----:|:----:|:----:|

**Trennzeichen**: *Semikolon*: `;`

### Jahrgang 5

In [33]:
kurse_mit_limits_jgst_5 = {
    'Liebe auf den zweiten Blick - (Wieder)verwenden statt wegwerfen': 32,
    'Scifi Schreibwerkstatt - Zukunftsideen oder "Wie leben wir in 100 Jahren?"': 16,
    'Singen, Tanzen und Schauspiel – Ein Performance-Workshop zum Nachdenken ': 16,
    'Schräge Vögel (und andere Tiere)': 32,
    'Yes, ve gan!': 32,
    "Kein' Scheiß - Wir kritzeln die Toiletten weiß!": 32,
    'Gesundheitssport und Schuluniform': 0,
    '"Nachhaltige Entwicklung" - Die Zukunftswerkstatt': 0
}

wahlergebnisse_jgst_5 = "./data/wahlergebnisse/Wahlergebnisse_jgst_5_ohne_jahrgangsuebergreifend.csv"

klassenlisten_jgst_5 = ["ge100041-5.1.1.csv", "ge100041-5.1.2.csv", "ge100041-5.1.3.csv",
                        "ge100041-5.2.1.csv", "ge100041-5.2.2.csv", "ge100041-5.2.3.csv"]
                
vollst_sus_listen_jgst_5 = ["./data/klasselisten/" + filename for filename in klassenlisten_jgst_5]

### Jahrgang 6

In [34]:
kurse_mit_limits_jgst_6 = {}

wahlergebnisse_jgst_6 = ""

klassenlisten_jgst_6 = []

vollst_sus_listen_jgst_6 = ["./data/klasselisten/" + filename for filename in klassenlisten_jgst_6]

### Jahrgang 7

In [35]:
kurse_mit_limits_jgst_7 = {}

wahlergebnisse_jgst_7 = ""

klassenlisten_jgst_7 = []

vollst_sus_listen_jgst_7 = ["./data/klasselisten/" + filename for filename in klassenlisten_jgst_7]

### Jahrgang 8

In [36]:
kurse_mit_limits_jgst_8 = {}

wahlergebnisse_jgst_8 = ""

klassenlisten_jgst_8 = []

vollst_sus_listen_jgst_8 = ["./data/klasselisten/" + filename for filename in klassenlisten_jgst_8]

### Jahrgang 9

In [37]:
kurse_mit_limits_jgst_9 = {
    "Artenvielfalt":18,
    "Nachhaltigkeit im Kleiderschrank":19,
    "Yes, ve-gan! Vegan/Vegetarisch - die Ernährung der Zukunft?":19,
    "Das zerbrechliche Paradies":19,
    "Escape, aber richtig!":19,
    "Nachhaltig Skifahren?!":34,
    "Nachhaltiges Leben und Herstellung eigener Pflegeprodukte":19,
    "Bau eines nachhaltigen Schülerkiosk":9,
    "Grüne Moderne. Die neue Sicht auf Pflanzen": 19,
    "Freies Projekt":19,
    "Gesundheitssport und Schuluniform":19
}

wahlergebnisse_jgst_9 = "./data/wahlergebnisse/Wahlergebnisse_jgst_9.csv"

klassenlisten_jgst_9 = ["ge100041-9.1.1.csv", "ge100041-9.1.2.csv", "ge100041-9.1.3.csv",
                        "ge100041-9.2.1.csv", "ge100041-9.2.2.csv", "ge100041-9.2.3.csv"]
                
vollst_sus_listen_jgst_9 = ["./data/klasselisten/" + filename for filename in klassenlisten_jgst_9]

## Ausführen der Zuteilung

In [38]:
verteilung = Verteilung(wahlergebnisse_jgst_9, 1, kurse_mit_limits_jgst_9)# , *vollst_sus_listen_jgst_9)

verteilung.erstellen(rest_zufaellig_verteilen=True)

verteilung.to_csv('./data/output/output_jgst_9.csv')

adj_mat = verteilung.make_adj_matrix()

verteilung.info()

136 Schüler:innen von 136 Schüler:innen verteilt.
--------------------------------------------------
Belegung:
Kurs
Artenvielfalt                                                  18
Bau eines nachhaltigen Schülerkiosk                             9
Das zerbrechliche Paradies                                     19
Escape, aber richtig!                                          19
Freies Projekt                                                 10
Gesundheitssport und Schuluniform                               3
Grüne Moderne. Die neue Sicht auf Pflanzen                      2
Nachhaltiges Leben und Herstellung eigener Pflegeprodukte      18
Nachhaltigkeit im Kleiderschrank                               19
Yes, ve-gan! Vegan/Vegetarisch - die Ernährung der Zukunft?    19
Name: Klasse, dtype: int64
--------------------------------------------------
Plätze übrig:  77


/var/folders/c2/mjqd965550vcl_p29gbscvcr0000gn/T/ipykernel_54666/3840033212.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_adj_matrix[f"{kurs}_platz_{i+1}"] = df_kurs_gewählt.astype(int)
/var/folders/c2/mjqd965550vcl_p29gbscvcr0000gn/T/ipykernel_54666/3840033212.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_adj_matrix[f"{kurs}_platz_{i+1}"] = df_kurs_gewählt.astype(int)
/var/folders/c2/mjqd965550vcl_p29gbscvcr0000gn/T/ipykernel_54666/3840033212.py:71: PerformanceWarning: DataFrame is highly fragmented.  This i